In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [3]:
df1 = pd.read_csv("/Users/iclon/Downloads/meneame_scraped_final.csv", encoding="utf-8")
df2 = pd.read_csv("/Users/iclon/Downloads/meneame_scraped_final_2.csv", encoding="utf-8")
df3 = pd.read_csv("/Users/iclon/Downloads/meneame_scraped_final_3.csv", encoding="utf-8")

In [4]:
df = pd.concat([df1,df2,df3], ignore_index=True)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374282 entries, 0 to 374281
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   news_id          374282 non-null  int64 
 1   title            374282 non-null  object
 2   content          374265 non-null  object
 3   meneos           374282 non-null  int64 
 4   clicks           374282 non-null  int64 
 5   karma            374282 non-null  int64 
 6   positive_votes   374282 non-null  int64 
 7   anonymous_votes  374282 non-null  int64 
 8   negative_votes   374282 non-null  int64 
 9   category         374282 non-null  object
 10  comments         374282 non-null  int64 
 11  published_date   374282 non-null  object
 12  author           287091 non-null  object
 13  source           374282 non-null  object
 14  source_link      374282 non-null  object
 15  provincia        374282 non-null  object
 16  comunidad        374282 non-null  object
 17  scraped_da

In [6]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [7]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'comunidad'

In [8]:
# Diccionario de provincias-ccaa
PROVINCIAS_COMUNIDADES = {
    "Andalucía": {
        "provincias": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"]
    },
    "Aragón": {
        "provincias": ["Huesca", "Teruel", "Zaragoza"]
    },
    "Asturias": {
        "provincias": ["Asturias"]
    },
    "Islas Baleares": {
        "provincias": ["Illes Balears"],
        "equivalencias": {
            "Mallorca": "Illes Balears",
            "Menorca": "Illes Balears",
            "Cabrera": "Illes Balears",
            "Ibiza": "Illes Balears",
            "Formentera": "Illes Balears"
        }
    },
    "Canarias": {
        "provincias": ["Las Palmas", "Santa Cruz de Tenerife"],
        "equivalencias": {
            "Gran Canaria": "Las Palmas",
            "Lanzarote": "Las Palmas",
            "Fuerteventura": "Las Palmas",
            "La Graciosa": "Las Palmas",
            "Tenerife": "Santa Cruz de Tenerife",
            "La Gomera": "Santa Cruz de Tenerife",
            "El Hierro": "Santa Cruz de Tenerife",
            "La Palma": "Santa Cruz de Tenerife"
        }
    },
    "Cantabria": {
        "provincias": ["Cantabria"]
    },
    "Castilla La Mancha": {
        "provincias": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"]
    },
    "Castilla y León": {
        "provincias": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"]
    },
    "Cataluña": {
        "provincias": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "equivalencias": {
            "Gerona": "Girona",
            "Lérida": "Lleida"
        }
    },
    "Extremadura": {
        "provincias": ["Badajoz", "Cáceres"]
    },
    "Galicia": {
        "provincias": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
        "equivalencias": {
            "Coruña": "A Coruña"
        }
    },
    "Madrid": {
        "provincias": ["Madrid"]
    },
    "Murcia": {
        "provincias": ["Murcia"]
    },
    "Navarra": {
        "provincias": ["Navarra"]
    },
    "La Rioja": {
        "provincias": ["La Rioja"]
    },
    "País Vasco": {
        "provincias": ["Araba/Álava", "Bizkaia/Vizcaya", "Gipuzkoa/Guipúzcoa"],
        "equivalencias": {
            "Araba": "Araba/Álava",
            "Álava": "Araba/Álava",
            "Bizkaia": "Bizkaia/Vizcaya",
            "Vizcaya": "Bizkaia/Vizcaya",
            "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
            "Guipúzcoa": "Gipuzkoa/Guipúzcoa"
        }
    },
    "Comunidad Valenciana": {
        "provincias": ["Alicante/Alacant", "Castelló/Castellón", "València/Valencia"],
        "equivalencias": {
            "Alicante": "Alicante/Alacant",
            "Alacant": "Alicante/Alacant",
            "Castellón": "Castelló/Castellón",
            "Castelló": "Castelló/Castellón",
            "Valencia": "València/Valencia",
            "València": "València/Valencia"
        }
    },
    "Ceuta": {
        "provincias": ["Ceuta"]
    },
    "Melilla": {
        "provincias": ["Melilla"]
    }
}

In [9]:
# Función para asignar provincias y ccaa del title/content
def asignar_provincia_comunidad(row):
    title = str(row["title"]) if pd.notna(row["title"]) else ""
    content = str(row["content"]) if pd.notna(row["content"]) else ""

    for comunidad, datos in PROVINCIAS_COMUNIDADES.items():
        provincias = datos["provincias"]
        equivalencias = datos.get("equivalencias", {})

        for provincia in provincias:
            if provincia in title or provincia in content:
                return {"provincia": provincia, "comunidad": comunidad}

        for variante, provincia_estandar in equivalencias.items():
            if variante in title or variante in content:
                return {"provincia": provincia_estandar, "comunidad": comunidad}

    return {"provincia": "Desconocido", "comunidad": "Desconocido"}

In [10]:
# Aplicar la función fila por fila
df[["provincia", "comunidad"]] = df.apply(asignar_provincia_comunidad, axis=1).apply(pd.Series)

In [11]:
df["comunidad"].value_counts()

comunidad
Desconocido             239538
Madrid                   14371
Andalucía                 6521
Comunidad Valenciana      5529
Cataluña                  5432
Castilla y León           3883
Castilla La Mancha        1585
Galicia                   1564
Aragón                    1363
Canarias                  1319
Murcia                    1232
Asturias                  1136
Islas Baleares            1036
Navarra                    539
País Vasco                 531
Extremadura                488
Cantabria                  407
Ceuta                      240
Melilla                    158
La Rioja                   146
Name: count, dtype: int64

In [12]:
df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)

df['provincia'] = df['provincia'].replace('Desconocido', np.nan)



# Procesamiento de Categorias


In [13]:
news_category_map = {
    "Geopolítica y Sociedad": ["EEUU", "MADRID", "Rusia", "Japón", "México", "Venezuela", "Cataluña", "Galiza", "Andalucía", "Balears", "Canarias", "CastillaLeón", "deutschland", "Barcelona", "Galicia", "Cantabria", "Aragón", "Málaga", "Roma", "INTERNAC", "Africa", "OrienteMedio", "Cuba"],
    "Movimientos políticos": ["politica", "corruPPción", "corruptcion", "corrupcion", "EleccionesE", "Podemos", "República", "RentaBásica", "Cayetanos", "LuchaDClases", "Anarquismo", "DDHH", "Autonomos", "Justicia7291", "Tribunales", "actualidad", "ZasSmeame", "Campañas", "Fachéame", "VIOLENCIA_G", "prensa", "borbones", "KKKantiPDMS", "AEDE", "procescatalà", "RebeliónMnm", "DchoLaboral", "Bulos", "Desinfórmame", "Madriléame", "Begoñéame"],
    "Conflictos y Guerra": ["Palestina", "Israel", "Siria", "GuerraCivilñ", "ayudaucrania", "Rescates"],
    "Tecnología y Ciencia": ["tecnología", "Informática", "GNU_LINUX", "linux", "startup", "Criptomoneda", "bitcoin", "SectorTIC", "Meteorología", "Física", "genetica", "Neurociencia", "Debian", "retrocomp", "hacking", "Metamaterial", "Micromundo", "atomo", "Impresión3D", "Drones", "netsec", "RaspberryPi", "SysDevs", "ciencia", "astronomia", "ASTRONÁUTICA", "softlibre", "Ubuntu", "android", "smartphone", "Firefox", "Blender", "IOOcia", "Marte", "Visualdata", "INVENTOS", "Numismática", "FÓSILES"],
    "Salud y Medicina": ["salud", "medicina", "Nutrición", "sanidad", "PREVENCIÓN", "HOMEOPATÍA", "VacunaGate", "vacúname", "SANIDAD", "Alimentación", "ebola", "Coronavirus"],
    "Gastronomía": ["Cachopos", "cervecéame"],
    "Negocios y Economía": ["Economía", "banca", "subvenciones", "CrisEnergéti"],
    "Entretenimiento y Cultura": ["ocio", "Cine", "Series", "Videojuegos", "Fotografía", "Música", "cinéfilos", "FolkloreÑ", "Cómics", "Webcomics", "ArteyMercado", "Arte", "Ilustracion", "clásica", "PALABROS", "Covers", "cultura", "Eurovision", "Poesia", "Bushido", "Arquitectura", "rol", "Podcasteros", "Viajar", "CiFi", "Gastronomia", "Clicómics", "Cocíname", "curiosidades", "acapela", "twitter", "reddit", "museo", "Relatocorto", "Autorrelatos", "Reportajes", "Entrevistas", "microrelatos", "Hazlotumismo", "Los12monos", "Pregúntame", "Recomiéndame", "LETREROS", "Escúchame", "Documentales", "STARWARS", "Ochenteando", "UHF", "Preguntas", "Fotomundo", "roleros", "metal", "B.S.O.", "TEMAZOS", "Comiñas", "MundoCelta", "Artículos", "sabíasque", "paléame"],
    "Deportes": ["deportes", "Natación", "Motociclismo", "ciclismo"],
    "Medioambiente y Energía": ["veganismo", "MenteAnimal", "Abandonos", "MUNDO_PROTE", "MAmbiente", "Cambioclima", "Ecología", "Energias", "sequía", "Naturaleza", "Botánica", "PLANTAS", "AgroInfo", "Vivienda", "Animales", "gatos", "Biologia", "Faros", "mascotas", "Cáñamo", "Arañas"],
    "Historia y Humanidades": ["Historia", "Mitología", "Arqueología", "Egiptologia", "Viñetas", "Hallazgos", "Asturias", "Jurídicas", "Etimologia", "Psicología", "Psicópatas", "Filosofía", "Efemérides", "Helénica", "Hemeroteca", "Fragmentos"],
    "Humor y Memes": ["Humor", "memes", "Sarcasmos", "Chorridolias", "Tendenciosos", "Ayuséame", "podríame", "Fakeame", "Fakejóo", "Idiocracia", "meneametoday", "TuMeme", "Cosasguays", "friki", "procrastrina", "NoMundoToday", "Memeneame", "Ardilléame", "Toréame", "OYOYOY", "gilipolleces", "chorradas"],
    "Transporte": ["aviacion", "aviones", "trenes", "DIRIGIBLES", "Submarino", "Motor", "Avionéame", "BARCOS", "Tractores", "movilidad", "alas"],
    "Educación": ["Educación", "idiomas", "Enseñanza", "Literatura", "Libros", "lenguas", "Matemáticas"],
    "Crimen": ["INV", "Gamonal", "Sectas", "Sucesos"],
    "Cuestiones Sociales": ["Asperger", "Crianza", "Drogolegas", "Feminismo", "LGBT", "Derecho", "SEXOLOGÍA", "mujeresenlah", "Abuso_Animal", "religion", "laicismo"]
}


In [14]:
def assign_category_from_text(text):
    assigned_categories = []

    text = text.lower() if isinstance(text, str) else ""

    for category, keywords in news_category_map.items():
        for keyword in keywords:
            if keyword.lower() in text:
                assigned_categories.append(category)
                break 

    return assigned_categories if assigned_categories else ["Miscelaneo"]

df["category"] = df["title"].apply(assign_category_from_text)

In [15]:
category_counts = df["category"].explode().value_counts()

category_counts_df = category_counts.reset_index()
category_counts_df.columns = ["Categoría", "Cantidad"]

import pandas as pd
from IPython.display import display

display(category_counts_df)

,Categoría,Cantidad
0,Miscelaneo,218639
1,Geopolítica y Sociedad,19405
2,Entretenimiento y Cultura,15482
3,Tecnología y Ciencia,6678
4,Crimen,5714
5,Historia y Humanidades,4584
6,Salud y Medicina,3934
7,Movimientos políticos,3802
8,Conflictos y Guerra,3789
9,Medioambiente y Energía,2804


In [16]:
df

,news_id,title,content,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,author,source,source_link,provincia,comunidad,scraped_date,full_story_link,user
0,4028534,Estudio realizado en Shenzhen muestra que los ...,Un programa piloto completado el año pasado en...,34,36,418,28,6,0,[Miscelaneo],11,2025-02-15 01:20:02,Gry,scmp.com,https://www.scmp.com/business/china-evs/articl...,NaN,NaN,2025-02-15,NaN,NaN
1,4028533,"Detienen al ladrón de Lucky, el perro de Navat...",La investigación ha permitido denunciar penalm...,60,315,425,41,19,1,[Miscelaneo],15,2025-02-15 00:45:03,Wolfgang,lavanguardia.com,https://www.lavanguardia.com/mascotas/20250213...,NaN,NaN,2025-02-15,NaN,NaN
2,4028513,"El padre de Marian Rojas, reconocido psiquiatr...","Las redes cancelan a Marian Rojas Estapé, serí...",88,294,453,53,35,1,[Miscelaneo],45,2025-02-15 00:15:02,Verdaderofalso,mundodeportivo.com,https://www.mundodeportivo.com/actualidad/2025...,NaN,NaN,2025-02-15,NaN,NaN
3,4028551,"Nieves Concostrina: ""No hubo ningún San Valent...","La historia oficiosa data del siglo III, en el...",107,258,416,60,47,4,[Crimen],28,2025-02-14 23:55:02,Charles_Dexter_Ward,cadenaser.com,https://cadenaser.com/nacional/2025/02/14/niev...,NaN,NaN,2025-02-15,NaN,NaN
4,4028478,Ofensiva israelí contra palestinos de Cisjorda...,La agencia de la ONU para los refugiados pales...,123,130,437,58,65,0,[Conflictos y Guerra],10,2025-02-14 23:15:02,onainigo,resumen.cl,https://resumen.cl/articulos/ofensiva-israeli-...,NaN,NaN,2025-02-15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287086,6,Instalar Linux en iPod Mini | GPL Tarragona,Enseña a instalar Linux en el iPod Mini.\n\nMu...,29,0,31,29,0,0,[Tecnología y Ciencia],0,2005-12-07 12:00:01,asappla,gpltarragona.org,http://www.gpltarragona.org/node/view/395,Tarragona,Cataluña,2025-02-16,NaN,NaN
287087,4,Entrevista de El Mundo a Jose Antonio Marina,2. ¿Es la ética la filosofía del futuro?\nSi. ...,18,0,22,18,0,0,[Miscelaneo],0,2005-12-07 11:30:01,gabrielperezs,el-mundo.es,http://www.el-mundo.es/encuentros/invitados/20...,NaN,NaN,2025-02-16,NaN,NaN
287088,3,Prova xfce 4.2.3!,Ara que els de repositoris de totes les distri...,34,0,13,34,0,0,[Miscelaneo],0,2005-12-07 10:30:01,paurullan,xfce.org,http://www.xfce.org/release_notes/4.2.3.1_chan...,NaN,NaN,2025-02-16,NaN,NaN
287089,2,Los 84 errores de noxtrum,"Pues sí, noxtrum, el buscador de TPI y Telefón...",61,0,11,61,0,0,[Miscelaneo],5,2005-12-07 09:25:02,FrIkI,pacoros.net,http://www.pacoros.net/diario/2005/12/07/los-8...,NaN,NaN,2025-02-16,NaN,NaN


In [17]:
df.to_csv("meneame_procesado.csv", index=False, encoding="utf-8")


In [2]:

#Organizar cada página y funcionalidad en funciones , una estructura modular..


SyntaxError: invalid syntax (2334867191.py, line 1)

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


#Cargar datos en session_state para evitar recargas innecesarias
if "data" not in st.session_state:
    st.session_state["data"] = None

if "theme" not in st.session_state:
    st.session_state["theme"] = "Claro"

#Función para la página de inicio
def home():
    st.title("🏠 Página de Inicio")
    st.write("Bienvenido a la aplicación de Streamlit.")
    st.write("Utiliza la barra lateral para navegar entre las secciones.")

#Función para la página de análisis de datos
def data_analysis():
    st.title("📊 Análisis de Datos")
    st.write("Carga y analiza un archivo CSV.")

    file = st.file_uploader("📂 Sube un archivo CSV", type=["csv"])
    if file is not None:
        df = pd.read_csv(file)
        st.session_state["data"] = df  # Guardar en session_state
        st.write("🔍 Vista previa del dataset:")
        st.write(df.head())
        st.write("📊 Estadísticas:")
        st.write(df.describe())
        st.write("📈 Distribución de datos:")
        st.bar_chart(df.select_dtypes(include=[np.number]))

#Función para la página de predicción con un modelo real
def prediction():
    st.title("🤖 Predicciones")

    if st.session_state["data"] is not None:
        df = st.session_state["data"]
        if df.shape[1] < 2:
            st.error("⚠️ El dataset necesita al menos dos columnas para entrenar un modelo.")
            return

        X = df.iloc[:, :-1].values  # Características (todas menos la última columna)
        y = df.iloc[:, -1].values   # Etiqueta (última columna)

        normalize = st.checkbox("Normalizar datos")
        if normalize:
            scaler = StandardScaler()
            X = scaler.fit_transform(X)

        model = LinearRegression()
        model.fit(X, y)

        input_value = st.number_input("✍ Ingresa un valor para predecir:", value=0.0, step=0.1)

        if st.button("Predecir"):
            try:
                pred = model.predict(np.array([[input_value]]))
                st.success(f"📈 Predicción del modelo: {pred[0]:.2f}")

                # Gráfico de dispersión con la recta de regresión
                fig, ax = plt.subplots()
                ax.scatter(X, y, color="blue", label="Datos")
                ax.plot(X, model.predict(X), color="red", label="Regresión Lineal")
                ax.set_xlabel("X")
                ax.set_ylabel("Y")
                ax.legend()
                st.pyplot(fig)
            except Exception as e:
                st.error(f"❌ Error en la predicción: {e}")
    else:
        st.warning("🔹 Primero sube un archivo en la sección de Análisis de Datos.")

#Función para la página de configuración
def settings():
    st.title("⚙️ Configuración")
    theme = st.selectbox("🎨 Elige un tema", ["Claro", "Oscuro"])
    st.session_state["theme"] = theme
    st.write(f"Has seleccionado: {theme}")

#Menú de navegación en la barra lateral
st.sidebar.title("📌 Navegación")
page = st.sidebar.radio("Selecciona una página:", ["Inicio", "Análisis de Datos", "Predicción", "Configuración"])

#Ejecutar la función de la página seleccionada
pages = {"Inicio": home, "Análisis de Datos": data_analysis, "Predicción": prediction, "Configuración": settings}
pages[page]()  # Llama a la función correspondiente.


2025-02-26 15:25:59.036 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 15:25:59.038 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-02-26 15:25:59.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 15:25:59.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 15:25:59.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 15:25:59.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_c